### Score Calculation and Game Simulation

In [197]:
import pandas as pd

In [198]:
df = pd.read_csv('final_df.csv')

In [199]:
df.columns

Index(['team', 'conference', 'tempo_adj', 'off_eff_adj', 'def_eff_adj',
       'Off-eFG%', 'Off-TO%', 'Off-OR%', 'Off-FTRate', 'Def-eFG%', 'Def-TO%',
       'Def-OR%', 'Def-FTRate', 'EffHgt', 'C-Hgt', 'Bench', 'Continuity',
       'Experience', '3P%', 'FT%', '3PA%', 'SOS'],
      dtype='object')

In [200]:
from sklearn.preprocessing import MinMaxScaler

# List of stats we want to normalize
stats_to_normalize = [
    'tempo_adj', 'off_eff_adj', 'def_eff_adj',
       'Off-eFG%', 'Off-TO%', 'Off-OR%', 'Off-FTRate', 'Def-eFG%', 'Def-TO%',
       'Def-OR%', 'Def-FTRate', 'EffHgt', 'C-Hgt', 'Bench', 'Continuity',
       'Experience', '3P%', 'FT%', '3PA%', 'SOS'
]

# Apply Min-Max Scaling
scaler = MinMaxScaler()
df[stats_to_normalize] = scaler.fit_transform(df[stats_to_normalize])

In [201]:
df

,team,conference,tempo_adj,off_eff_adj,def_eff_adj,Off-eFG%,Off-TO%,Off-OR%,Off-FTRate,Def-eFG%,...,Def-FTRate,EffHgt,C-Hgt,Bench,Continuity,Experience,3P%,FT%,3PA%,SOS
0,Abilene Christian,WAC,0.623377,0.344330,0.402532,0.358824,0.614173,0.485597,0.623529,0.544828,...,1.000000,0.350000,0.342105,0.627273,0.466488,0.372928,0.192857,0.500000,0.157738,0.321555
1,Air Force,MWC,0.318182,0.364948,0.630380,0.535294,0.637795,0.251029,0.517647,0.737931,...,0.519637,0.533333,0.552632,0.390909,0.650134,0.494475,0.492857,0.189076,0.794643,0.511661
2,Akron,MAC,0.824675,0.618557,0.478481,0.782353,0.346457,0.613169,0.101961,0.358621,...,0.489426,0.000000,0.144737,0.772727,0.320375,0.386740,0.628571,0.617647,0.747024,0.245230
3,Alabama,SEC,1.000000,0.929897,0.270886,0.900000,0.354331,0.773663,0.709804,0.227586,...,0.462236,0.850000,0.605263,0.736364,0.592493,0.825967,0.592857,0.525210,0.794643,0.995760
4,Alabama A&M,SWAC,0.805195,0.228866,0.721519,0.235294,0.724409,0.716049,0.650980,0.710345,...,0.915408,0.466667,0.473684,0.915152,0.455764,0.237569,0.250000,0.294118,0.622024,0.023675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,Wright St.,Horz,0.532468,0.562887,0.675949,0.852941,0.385827,0.427984,0.188235,0.613793,...,0.504532,0.266667,0.210526,0.506061,0.628686,0.552486,0.857143,0.420168,0.500000,0.254417
360,Wyoming,MWC,0.357143,0.463918,0.481013,0.564706,0.582677,0.506173,0.317647,0.517241,...,0.688822,0.483333,0.434211,0.745455,0.131367,0.494475,0.614286,0.252101,0.580357,0.571025
361,Xavier,BE,0.623377,0.674227,0.281013,0.723529,0.291339,0.325103,0.576471,0.413793,...,0.317221,0.533333,0.407895,0.321212,0.351206,0.977901,0.850000,0.819328,0.511905,0.695406
362,Yale,Ivy,0.545455,0.694845,0.445570,0.800000,0.165354,0.691358,0.333333,0.331034,...,0.359517,0.333333,0.289474,0.509091,0.706434,0.466851,0.800000,0.609244,0.336310,0.272085


In [202]:
def calculate_team_rating(row):
    # Calculate Adjusted Efficiency Margin (normalized)
    adj_em = row["off_eff_adj"] - row["def_eff_adj"]

    # Weights for each stat
    weights = {
        "AdjEM": 0.40,   # Overall efficiency
        "SOS": 0.20,     # Strength of Schedule
        "tempo_adj": 0.05,  # Tempo/Pace
        "Off-eFG%": 0.07,  # Offensive Shooting Efficiency
        "Def-eFG%": 0.07,  # Defensive Shooting Efficiency
        "Off-TO%": -0.05,  # Turnovers (negative impact)
        "Def-TO%": 0.05,   # Defensive Turnovers Forced
        "Off-OR%": 0.05,   # Offensive Rebounding
        "Def-OR%": -0.05,  # Opponent Offensive Rebounding (lower is better)
        "Off-FTRate": 0.03,  # Getting to the FT line
        "Def-FTRate": -0.03,  # Opponent getting to the FT line (lower is better)
        "Experience": 0.10,  # Veteran teams perform better
        "Continuity": 0.05,  # Returning players help chemistry
        "Bench": 0.03,  # Bench depth
        "3P%": 0.05,   # 3-point shooting
        "FT%": 0.03,   # Free throw shooting
        "3PA%": 0.03,  # Volume of 3-pointers taken
        "EffHgt": 0.03,  # Effective height
        "C-Hgt": 0.00  # Center height
    }

    # Compute team rating as a weighted sum
    rating = (
        weights["AdjEM"] * adj_em +
        weights["SOS"] * row["SOS"] +
        weights["tempo_adj"] * row["tempo_adj"] +
        weights["Off-eFG%"] * row["Off-eFG%"] +
        weights["Def-eFG%"] * row["Def-eFG%"] +
        weights["Off-TO%"] * row["Off-TO%"] +
        weights["Def-TO%"] * row["Def-TO%"] +
        weights["Off-OR%"] * row["Off-OR%"] +
        weights["Def-OR%"] * row["Def-OR%"] +
        weights["Off-FTRate"] * row["Off-FTRate"] +
        weights["Def-FTRate"] * row["Def-FTRate"] +
        weights["Experience"] * row["Experience"] +
        weights["Continuity"] * row["Continuity"] +
        weights["Bench"] * row["Bench"] +
        weights["3P%"] * row["3P%"] +
        weights["FT%"] * row["FT%"] +
        weights["3PA%"] * row["3PA%"] +
        weights["EffHgt"] * row["EffHgt"] +
        weights["C-Hgt"] * row["C-Hgt"]
    )

    return rating

In [203]:
df["team_rating"] = df.apply(calculate_team_rating, axis=1)

In [204]:
final_stat_rating = df[['team','team_rating']].sort_values(by='team_rating',ascending = False)

Now that we have the final rating for all kenpom stats we can add rating boosts and matchup boosts

#### Rating and Matchup Boosts

In [207]:
def rating_boosts (row, opponent):
    boost_rating = 0

    #Tempo Boost
    tempo_diff = row["tempo_adj"] - opponent["tempo_adj"]
    if tempo_diff > 0.10:  
        boost_rating += 0.02

    return boost_rating

    

In [208]:
#Sim Games
import numpy as np

def simulate_game (team_name1, team_name2, df):
    team1_row = df[df["team"] == team_name1].iloc[0]
    team2_row = df[df['team'] == team_name2].iloc[0]

    team1_score = calculate_team_rating(team1_row)
    team2_score = calculate_team_rating(team2_row)

    team1_score += rating_boosts(team1_row,team2_row)
    team2_score += rating_boosts(team1_row,team2_row)

    rating_diff = (team1_score - team2_score) * 6  # Scale factor to adjust win probability distribution
    win_prob_team1 = 1 / (1 + np.exp(-rating_diff))  

    # Simulate game outcome
    random_number = np.random.rand()
    if random_number < win_prob_team1:
        winner = team_name1
    else: 
        winner = team_name2
    return winner
    

#### Bracket Dictionary (2023 for now)

In [210]:
def load_bracket():
    bracket = {
        "East": [
            (1, "Purdue") , (16, "Fairleigh Dickinson"),
            (8, "Memphis"), (9, "Florida Atlantic"),
            (5, "Duke"), (12, "Oral Roberts"),
            (4, "Tennessee"), (13, "Louisiana"),
            (6, "Kentucky"), (11, "Providence"),
            (3, "Kansas St."), (14, "Montana St."),
            (7, "Michigan St."), (10, "USC"),
            (2, "Marquette"), (15, "Vermont")
        ],
        "West": [
            (1, "Kansas"), (16, "Howard"),
            (8, "Arkansas"), (9, "Illinois"),
            (5, "Saint Mary's"), (12, "VCU"),
            (4, "Connecticut"), (13, "Iona"),
            (6, "TCU"), (11, "Arizona St."),
            (3, "Gonzaga"), (14, "Grand Canyon"),
            (7, "Northwestern"), (10, "Boise St."),
            (2, "UCLA"), (15, "UNC Asheville")
        ],
        "South": [
            (1, "Alabama"), (16, "Texas A&M Corpus Chris"),
            (8, "Maryland"), (9, "West Virginia"),
            (5, "San Diego St."), (12, "Charleston"),
            (4, "Virginia"), (13, "Furman"),
            (6, "Creighton"), (11, "N.C. State"),
            (3, "Baylor"), (14, "UC Santa Barbara"),
            (7, "Missouri"), (10, "Utah St."),
            (2, "Arizona"), (15, "Princeton")
        ],
        "Midwest": [
            (1, "Houston"), (16, "Northern Kentucky"),
            (8, "Iowa"), (9, "Auburn"),
            (5, "Miami FL"), (12, "Drake"),
            (4, "Indiana"), (13, "Kent St."),
            (6, "Iowa St."), (11, "Pittsburgh"),
            (3, "Xavier"), (14, "Kennesaw St."),
            (7, "Texas A&M"), (10, "Penn St."),
            (2, "Texas"), (15, "Colgate")
        ]
    }
    return bracket

In [211]:
def print_bracket(bracket, old_bracket):
    
    for region in old_bracket.keys():
        print(f"🌎 {region} Region:")
        
        # Extract winners from the current bracket
        current_winners = {team[1] for team in bracket[region]}  # Store only the team names in a set
        
        for matchup in old_bracket[region]:  # Loop through the previous round's matchups
            seed, team = matchup
            
            if team in current_winners:  
                print(f"{seed} {team} ✅")
            else: 
                print(f"{seed} {team} ❌")
        
        print("\n" + "-"*30)

In [212]:
def print_final_four(teams, winners):
    print("\n🏆 Final Four 🏆\n")
    if teams[0][0][1] == winners[0][1]:
        print(teams[0][0][0], teams[0][0][1], "✅")
        print(teams[1][0][0], teams[1][0][1], "❌")
    else:
        print(teams[0][0][0], teams[0][0][1], "❌")
        print(teams[1][0][0], teams[1][0][1], "✅")
    print('\n')
    if teams[2][0][1] == winners[1][1]:
        print(teams[2][0][0], teams[2][0][1], "✅")
        print(teams[3][0][0], teams[3][0][1], "❌")
    else:
        print(teams[2][0][0], teams[2][0][1], "❌")
        print(teams[3][0][0], teams[3][0][1], "✅")
    print("\n" + "-"*30)

In [213]:
def print_chip(teams, champ):
    print("\n🏆 National Championship 🏆\n")
    if teams[0][1] == champ:
        print(teams[0][0],teams[0][1], "✅")
        print(teams[1][0],teams[1][1], "❌")
    else:
        print(teams[0][0],teams[0][1], "❌")
        print(teams[1][0],teams[1][1], "✅")
    print("\n" + "-"*30)
    print("\n🏀🏆 Champion 🏆🏀\n")
    print(champ)

In [214]:
import copy
def madness_sim(bracket, df):
    print("🏀 NCAA March Madness Bracket 🏀\n")
    regions = ['East', 'West', 'South', 'Midwest']
    
    round_names = ["First Round", "Round of 32", "Sweet 16", "Elite 8"]
    round_winners_list = []
    for round_index, round_name in enumerate(round_names):
        print(f"\n🏀 {round_name} 🏀\n")
        old_bracket = copy.deepcopy(bracket)
        for region in regions:
            winners_list = []
            step = 16 // (2 ** round_index)
    
            for i in range(0, step, 2):
                team1 = bracket[region][i][1]
                team2 = bracket[region][i+1][1]
                winner = simulate_game(team1, team2, df)
    
                if team1 == winner:
                    winners_list.append(bracket[region][i])
                else:
                    winners_list.append(bracket[region][i+1])
                
            
            bracket[region] = winners_list  # Update the bracket
            round_winners_list.append(winners_list)
        print_bracket(bracket, old_bracket)
            
    final_four = []
    for region in regions:
        final_four.append(bracket[region])
    #Final Four
    winners_list = []
    for i in range(0,4,2):
        team1 = final_four[i][0][1]
        team2 = final_four[i+1][0][1]
        winner = simulate_game(team1,team2, df)
        if team1 == winner:
            winners_list.append(final_four[i][0])
        else:
            winners_list.append(final_four[i+1][0])
    print_final_four(final_four, winners_list)

    #Championship
    champion = simulate_game(winners_list[0][1], winners_list[1][1], df)
    print_chip(winners_list, champion)

In [215]:
madness_sim(load_bracket(), df)

🏀 NCAA March Madness Bracket 🏀


🏀 First Round 🏀

🌎 East Region:
1 Purdue ❌
16 Fairleigh Dickinson ✅
8 Memphis ✅
9 Florida Atlantic ❌
5 Duke ✅
12 Oral Roberts ❌
4 Tennessee ✅
13 Louisiana ❌
6 Kentucky ✅
11 Providence ❌
3 Kansas St. ✅
14 Montana St. ❌
7 Michigan St. ❌
10 USC ✅
2 Marquette ✅
15 Vermont ❌

------------------------------
🌎 West Region:
1 Kansas ✅
16 Howard ❌
8 Arkansas ❌
9 Illinois ✅
5 Saint Mary's ❌
12 VCU ✅
4 Connecticut ✅
13 Iona ❌
6 TCU ✅
11 Arizona St. ❌
3 Gonzaga ✅
14 Grand Canyon ❌
7 Northwestern ✅
10 Boise St. ❌
2 UCLA ✅
15 UNC Asheville ❌

------------------------------
🌎 South Region:
1 Alabama ✅
16 Texas A&M Corpus Chris ❌
8 Maryland ✅
9 West Virginia ❌
5 San Diego St. ✅
12 Charleston ❌
4 Virginia ❌
13 Furman ✅
6 Creighton ✅
11 N.C. State ❌
3 Baylor ❌
14 UC Santa Barbara ✅
7 Missouri ✅
10 Utah St. ❌
2 Arizona ✅
15 Princeton ❌

------------------------------
🌎 Midwest Region:
1 Houston ✅
16 Northern Kentucky ❌
8 Iowa ❌
9 Auburn ✅
5 Miami FL ❌
12 Drake ✅
4 Indiana